In [1]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import re
import time
import json
import csv
from glob import glob
from PIL import Image
import pickle
import shutil
from tqdm import tqdm
import chakin
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer
import gzip
from gensim import models as gmodel
import subprocess

True


In [ ]:
#download dataset
annotation_zip = kr.utils.get_file('/content/drive/My Drive/colab/coco_dataset/captions.zip',
                                          origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                          extract = False)

name_of_zip = 'train2014.zip'
image_zip = tf.keras.utils.get_file(name_of_zip,
                              cache_subdir=os.path.abspath('.'),
                                    origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                    extract = True)
  
name_of_zip = 'test2014.zip'
image_zip = kr.utils.get_file(name_of_zip,
                                    origin = 'http://images.cocodataset.org/zips/test2014.zip',
                                    extract = False)

name_of_zip = 'val2014.zip'
image_zip = tf.keras.utils.get_file(name_of_zip,
                              cache_subdir=os.path.abspath('.'),
                                    origin = 'http://images.cocodataset.org/zips/val2014.zip',
                                    extract = True)

In [ ]:
train_annotation_file = '/content/drive/My Drive/colab/coco_dataset/captions/annotations/captions_train2014.json'
test_annotation_file = '/content/drive/My Drive/colab/coco_dataset/captions/annotations/captions_test2014.json'
val_annotation_file = '/content/drive/My Drive/colab/coco_dataset/captions/annotations/captions_val2014.json'
train_incep_path = '/content/drive/My Drive/colab/coco_dataset/train_incep/'
test_incep_path = '/content/drive/My Drive/colab/coco_dataset/test_incep/'
val_incep_path = '/content/drive/My Drive/colab/coco_dataset/val_incep/'
train_img_path = '/content/drive/My Drive/colab/coco_dataset/train_img/'
test_img_path = '/content/drive/My Drive/colab/coco_dataset/test_img/'
val_img_path = '/content/drive/My Drive/colab/coco_dataset/val_img/'

In [ ]:
train_paths = glob('/content/drive/My Drive/colab/coco_dataset/train_img/*/*')
val_paths = glob('/content/drive/My Drive/colab/coco_dataset/val_img/*/*')

print(len(train_paths))
print(len(val_paths))

In [ ]:
#move unzipped files in subdirectories
os.mkdir(train_img_path)
os.mkdir(val_img_path)
for i in range(10):
    os.mkdir(train_img_path+str(i)+'/')
    os.mkdir(val_img_path+str(i)+'/')

def mv1(arg):
    i,f = arg
    i = i%10
    dest = train_img_path + str(i) + '/' + f[f.rfind('/')+1:]
    shutil.move(f,dest)

def mv2(arg):
    i,f = arg
    i = i%10
    dest = val_img_path + str(i) + '/' + f[f.rfind('/')+1:]
    shutil.move(f,dest)
    
lst = list(map(mv1,enumerate(train_paths)))
lst = list(map(mv2,enumerate(val_paths)))

In [ ]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (299, 299))
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

In [ ]:
#inception model for feature extraction
incep_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = incep_model.input
hidden_layer = incep_model.layers[-1].output

incep_features_extract_model = tf.keras.Model(new_input, hidden_layer)

print(incep_features_extract_model.layers[-1].output)

In [ ]:
#feature extraction
encode_train = train_paths

image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):
    batch_features = incep_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
        f = p.numpy().decode("utf-8")
        pof = train_incep_path + f[f.rfind('train_img')+10:f.rfind('.')]
        if not os.path.exists(pof):
            np.save(pof, bf.numpy())
            
encode_val = val_paths

image_dataset = tf.data.Dataset.from_tensor_slices(encode_val)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(16)

for img, path in tqdm(image_dataset):
    batch_features = incep_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                              (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
        f = p.numpy().decode("utf-8")
        pof = val_incep_path + f[f.rfind('val_img')+8:f.rfind('.')]
        if not os.path.exists(pof):
            np.save(pof, bf.numpy())

In [ ]:
train_img_paths = glob(train_img_path+'*/*')
val_img_paths = glob(val_img_path+'*/*')

print(len(train_img_paths))
print(len(val_img_paths))

train_incep_paths = glob(train_incep_path+'*/*npy')
val_incep_paths = glob(val_incep_path+'*/*npy')

print(len(train_incep_paths))
print(len(val_incep_paths))

In [ ]:
#analyzing tensor shapes after each layer of model.
features = tf.random.uniform((32,64,2048)) #example input tensor
print('features:',features.shape)
print()

print('encoder-')
features = tf.keras.layers.BatchNormalization()(features)
feat_mean = tf.reduce_mean(features, 1)
print('feat_mean:',feat_mean.shape)
h_state = tf.keras.layers.Dense(600)(feat_mean)
h_state = tf.nn.tanh(h_state)
print('h_state:',h_state.shape)
c_state = tf.keras.layers.Dense(600)(feat_mean)
c_state = tf.nn.tanh(c_state)
print('c_state:',c_state.shape)
fc = tf.keras.layers.Dense(300)
features = fc(features)
features = tf.nn.relu(features)
print('features:',features.shape)
print()

print('attention-')
W1 = tf.keras.layers.Dense(600)
W2 = tf.keras.layers.Dense(600)
V = tf.keras.layers.Dense(1)
hidden = tf.expand_dims(h_state, 1)
print('hidden:',hidden.shape)
W_features = W1(features)
W_hidden = W2(hidden)
print('W_features:',W_features.shape)
print('W_hidden:',W_hidden.shape)
W_add = W_features+W_hidden
print('W_add:',W_add.shape)
score = tf.nn.tanh(W_add)
print('score:',score.shape)
V_score = V(score)
print('V_score:',V_score.shape)
attn_weights = tf.nn.softmax(V_score)
print('attn_weights:',attn_weights.shape)
context_vector = attn_weights * features
print('context_vector:',context_vector.shape)
context_vector = tf.reduce_sum(context_vector, axis=1)
print('context_vector:',context_vector.shape)
print()

print('decoder-')
x  = tf.convert_to_tensor([tokenizer.word_index['<start>']] * 32, 1)
print('x:',x.shape)
x = tf.keras.layers.Embedding(vocab_size, embedding_dim,weights=[embed])(x)
print('x:',x.shape)
beta = tf.keras.layers.Dense(1)(h_state)
print('beta:',beta.shape)
beta = tf.nn.sigmoid(beta)
print('beta:',beta.shape)
context_vector = tf.multiply(beta, context_vector, name='selected_context')
print('context_vector:',context_vector.shape)
x_in = tf.concat([x,context_vector], axis=1)
print('x_in:',x_in.shape)
output,[c_state,h_state] = tf.keras.layers.LSTMCell(units=600,recurrent_initializer='glorot_uniform')(x,states=[c_state,h_state])
print('output:',output.shape)
print('c_state:',c_state.shape)
print('h_state:',h_state.shape)
h_state = tf.nn.dropout(h_state,0.5)
W_h = tf.keras.layers.Dense(300)(h_state)
print('W_h:',W_h.shape)
W_context = tf.keras.layers.Dense(300,use_bias=False)(context_vector)
print('W_context:',W_context.shape)
h_logits = W_h + W_context + x
print('h_logits:',h_logits.shape)
h_logits = tf.nn.tanh(h_logits)
h_logits = tf.nn.dropout(h_logits,0.5)
print('h_logits:',h_logits.shape)
out_logits = tf.keras.layers.Dense(vocab_size)(h_logits)
print('out_logits:',out_logits.shape)

In [ ]:
alpha_list = [attn_weights]*52
alpha_c = 0.01
alphas = tf.stack(alpha_list)
print('alphas:',alphas.shape)
alphas = tf.transpose(alphas, (1, 0, 2,3))
print('alphas:',alphas.shape)
alphas_all = tf.reduce_sum(alphas, 1)
print('alphas_all:',alphas_all.shape)
alpha_reg = alpha_c * tf.reduce_sum((52./64 - alphas_all) ** 2,1)
print('alpha_reg:',alpha_reg.shape)
alpha_reg = tf.reduce_mean(alpha_reg)
print('alpha_reg:',alpha_reg.shape)

In [ ]:
alpha_reg